In [9]:
from getpass import getpass
from arcgis.gis import GIS
import arcgis.features
import pandas as pd
from arcgis.geometry import Polygon
# Need to do conda install -c esri arcgis, which requires Python 3.8
# Run notebook with kernel created with:
# python -m ipykernel install --user --name arcgis --display-name "Python 3.8 (ArcGIS)"
# On Apple Silicon need Rosetta 2 emulator (see https://joelmccune.com/install-arcgis-python-api-on-apple-silicon/)
# and downgrade urllib3=1.26.0 until fall update of arcgis package
# TODO: update dependencies file for above

# Authenticate with ArcGIS Online
username = getpass("Entter ArcGIS username: ")

password = getpass("Enter ArcGIS password: ")

gis = GIS("https://www.arcgis.com", username, password)

# Define coordinates and area of interest
lat = 40.322965
lon = -74.728754
distance_in_miles = 0.25
# Convert distance to degrees (approx)
distance_in_degrees = distance_in_miles / 69.0 # 69 miles per degree at equator ... less at higher latitudes

# Define boundary square around the coordinates
min_lat = lat - distance_in_degrees
max_lat = lat + distance_in_degrees
min_lon = lon - distance_in_degrees
max_lon = lon + distance_in_degrees

# Polygon representing the area of interest
# https://developers.arcgis.com/documentation/common-data-types/geometry-objects.htm#:~:text=JSON%20examples-,A%202D%20polygon,-%7B%0A%20%20%22rings%22%3A%20%5B%0A%20%20%20%20%5B%0A%20%20%20%20%20%20%5B%2D97.06138%2C32.837%5D%2C%0A%20%20%20%20%20%20%5B%2D97.06133%2C32.836
area_of_interest = Polygon({
    "rings": [[[min_lon, min_lat], 
               [min_lon, max_lat], 
               [max_lon, max_lat], 
               [max_lon, min_lat], 
               [min_lon, min_lat]]],
    "spatialReference": {"wkid": 4326}
})



In [10]:
# Web map ID fo https://www.arcgis.com/home/search.html?restrict=false&sortField=relevance&sortOrder=desc&searchTerm=2705228b2b154d0a906ef7a54e533fac#content
webmap_id = "2705228b2b154d0a906ef7a54e533fac"

# Access item by ID
webmap_item = gis.content.get(webmap_id)

# https://developers.arcgis.com/python/guide/managing-your-content/
webmap_data = webmap_item.get_data()

webmap_data.keys()

dict_keys(['operationalLayers', 'baseMap', 'authoringApp', 'authoringAppVersion', 'initialState', 'spatialReference', 'version'])

In [11]:
webmap_data["operationalLayers"]

[{'id': '188f84d8736-layer-2',
  'opacity': 0.5,
  'title': 'Land Use Land Cover of New Jersey 2015 - Land Use 2015',
  'url': 'https://mapsdep.nj.gov/arcgis/rest/services/Features/Land_lu/MapServer/13',
  'itemId': '2e7d9704d28c420a85777c37ee4636ab',
  'layerType': 'ArcGISFeatureLayer',
  'popupInfo': {'popupElements': [{'type': 'fields'},
    {'type': 'attachments', 'displayType': 'auto'}],
   'showAttachments': True,
   'fieldInfos': [{'fieldName': 'OBJECTID',
     'isEditable': False,
     'label': 'OBJECTID',
     'visible': False},
    {'fieldName': 'ACRES',
     'format': {'digitSeparator': True, 'places': 2},
     'isEditable': True,
     'label': 'ACRES',
     'visible': True},
    {'fieldName': 'APPROVED',
     'isEditable': True,
     'label': 'APPROVED',
     'visible': True},
    {'fieldName': 'AREASQKM',
     'format': {'digitSeparator': True, 'places': 2},
     'isEditable': True,
     'label': 'Area (SQ KM)',
     'visible': True},
    {'fieldName': 'CHANGE15',
     'fo

In [12]:
# https://developers.arcgis.com/web-map-specification/objects/operationalLayers/
operational_layers = webmap_data["operationalLayers"]
print(f"Found {len(operational_layers)} operational layers.")
    
# Loop through each operational layer
print(f"Layer Name: {operational_layers[0].get('title')}, URL: {operational_layers[0].get('url')}")



Found 1 operational layers.
Layer Name: Land Use Land Cover of New Jersey 2015 - Land Use 2015, URL: https://mapsdep.nj.gov/arcgis/rest/services/Features/Land_lu/MapServer/13


In [13]:
# URL of the operational layer
layer_url = operational_layers[0].get('url')

# Access the layer
# https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html
layer = arcgis.features.FeatureLayer(layer_url)

# Query the layer for features that intersect with the area of interest
features = layer.query(geometry_filter=arcgis.geometry.filters.intersects(area_of_interest))

# Check features
print(f"Found {len(features)} features in the layer.")

# View the feature attributes
for feature in features:
    print(feature.attributes)


Found 18 features in the layer.
{'OBJECTID': 371923, 'ACRES': 2.43043655, 'LU15': 1130, 'LABEL15': 'RESIDENTIAL, SINGLE UNIT, LOW DENSITY', 'TYPE15': 'URBAN', 'IS15': 12.82607141, 'ISACRES15': 0.31172953000000003, 'DV15': 0, 'LU12': 1130, 'LABEL12': 'RESIDENTIAL, SINGLE UNIT, LOW DENSITY', 'TYPE12': 'URBAN', 'IS12': 10, 'ISACRES12': 0.24304366, 'DV12': 0, 'CHANGE15': 0, 'ISCHANGE15': 1, 'HU8': '02040105', 'STATUS': 'DRAFT', 'APPROVED': '01/28/19', 'COMID': None, 'PERMANENT_IDENTIFIER': None, 'FDATE': None, 'RESOLUTION': None, 'GNIS_ID': None, 'GNIS_NAME': None, 'AREASQKM': None, 'ELEVATION': None, 'REACHCODE': None, 'FTYPE': None, 'FCODE': None, 'FTYPE_DESCRIPTION': None, 'FCODE_DESCRIPTION': None, 'FTYPE_DISPLAY': None, 'NHD_FEATURECLASS': None, 'WATERBODY_NAME': None, 'FEATURE_ID': None, 'FEATURE_NAME': None, 'FEATURE_CLASS': None, 'LEVELELEV': None, 'GLOBALID': '{626BFC03-091E-4DB2-B848-74FD09E020E4}', 'SHAPE_Length': 1291.7361634679703, 'SHAPE_Area': 105869.3928495188, 'CREATED_USE

In [14]:
# Store in a dataframe
intersection_data = pd.DataFrame(columns=['Object_ID', 'Region', 'Area (sq meters)'])

# Loop through the features
# TODO: get only area within the area of interest and combine like regions
for feature in features:
    # Extract the object ID, region label, and area
    object_id = feature.attributes['OBJECTID']
    region_label = feature.attributes['LABEL15']
    region_area = feature.attributes['SHAPE_Area']
    intersection_data.loc[len(intersection_data)] = {'Object_ID': object_id, 'Region': region_label, 'Area (sq meters)': region_area}
    #intersection_data = intersection_data.append({'Region': region_label, 'Area (sq meters)': region_area}, ignore_index=True)

print(intersection_data)


    Object_ID                                             Region  \
0      371923              RESIDENTIAL, SINGLE UNIT, LOW DENSITY   
1      376716                    RESIDENTIAL, RURAL, SINGLE UNIT   
2      376735                    RESIDENTIAL, RURAL, SINGLE UNIT   
3      376744                    RESIDENTIAL, RURAL, SINGLE UNIT   
4      406231                           CROPLAND AND PASTURELAND   
5      409391                   AGRICULTURAL WETLANDS (MODIFIED)   
6      409392                   AGRICULTURAL WETLANDS (MODIFIED)   
7      414024            DECIDUOUS FOREST (10-50% CROWN CLOSURE)   
8      419968              DECIDUOUS FOREST (>50% CROWN CLOSURE)   
9      419983              DECIDUOUS FOREST (>50% CROWN CLOSURE)   
10     427126                                         PLANTATION   
11     427801  MIXED FOREST (>50% CONIFEROUS WITH >50% CROWN ...   
12     435138         MIXED DECIDUOUS/CONIFEROUS BRUSH/SHRUBLAND   
13     445893                          DECIDUOUS

Try the `overlaps` filter. Will check the object IDs online to see how these compare to what we can see.

In [15]:
# Query the layer for features that overlay the area of interest
features = layer.query(geometry_filter=arcgis.geometry.filters.overlaps(area_of_interest))

# Check features
print(f"Found {len(features)} features in the layer.")

# View the feature attributes
for feature in features:
    print(feature.attributes)

Found 11 features in the layer.
{'OBJECTID': 376716, 'ACRES': 8.35899046, 'LU15': 1140, 'LABEL15': 'RESIDENTIAL, RURAL, SINGLE UNIT', 'TYPE15': 'URBAN', 'IS15': 13.57034977, 'ISACRES15': 1.13434424, 'DV15': 0, 'LU12': 1140, 'LABEL12': 'RESIDENTIAL, RURAL, SINGLE UNIT', 'TYPE12': 'URBAN', 'IS12': 15, 'ISACRES12': 1.25384857, 'DV12': 0, 'CHANGE15': 0, 'ISCHANGE15': 0, 'HU8': '02040105', 'STATUS': 'DRAFT', 'APPROVED': '01/28/19', 'COMID': None, 'PERMANENT_IDENTIFIER': None, 'FDATE': None, 'RESOLUTION': None, 'GNIS_ID': None, 'GNIS_NAME': None, 'AREASQKM': None, 'ELEVATION': None, 'REACHCODE': None, 'FTYPE': None, 'FCODE': None, 'FTYPE_DESCRIPTION': None, 'FCODE_DESCRIPTION': None, 'FTYPE_DISPLAY': None, 'NHD_FEATURECLASS': None, 'WATERBODY_NAME': None, 'FEATURE_ID': None, 'FEATURE_NAME': None, 'FEATURE_CLASS': None, 'LEVELELEV': None, 'GLOBALID': '{62E4507C-7C38-4E3E-A035-74A7A2FD6CAF}', 'SHAPE_Length': 3555.2220169003554, 'SHAPE_Area': 364116.1679749123, 'CREATED_USER': 'DEP', 'CREATED_D

In [ ]:
# This spatial analysis tool *should* do what we want, however it requires an ArcGIS Online account under an organization
for feature in features:
    print(arcgis.features.analysis.summarize_within(sum_within_layer = area_of_interest, summary_layer = layer, group_by_field = 'LABEL15'))